# PHYS 641 Assignment 3
### Andrew V. Zwaniga
### (260843983) 



# Note: 

__I unzipped `LOSC_Event_tutorial.zip` in the directory where my code was made so this code should run as long as it is placed in the same directory as where you unzipped your copy of the data.__ 



## Problem 1: Finding gravitational waves in LIGO data 

We will investigate a data set obtained from the file `LOSC_Event_tutorial` that is publicly available at https://www.gw-openscience.org/data/. Contained is data from the Laser Interferometer Gravitational wave Observatory (LIGO) experiment that seeks to detect gravitational waves at Earth from exotic binary mergers like BH-BH binaries, BH-NS binaries, and NS-NS binaries that merge. 

The plan is to look at data from different events from the Hanford (H) and Livingston (L) observatories separately and come up with a unique noise model for each. Then we will loop over the four events for each observatory and search for the gravitational wave. The steps for working with the data will be as follows: 

### Part A: Discussion of the steps taken to estimate the noise matrix

#### (1) 
Load the raw data and apply a suitably chosen __window function__ on the samples. 

 - A __window function__ is one that goes to zero smoothly at either boundary of a finite interval. The purpose of this is to prevent effects of the periodicity of discrete Fourier transforms. By multiplying the window function by the data, the new dataset will go to zero on its boundary. 


#### (2) 
Compute the __power spectrum__ of the windowed data by taking the discrete Fourier transform of it. 

 - If $f(x)$ is a discrete function on datapoints $x=1,\dots,N$ and $F(k)$ is the discrete Fourier transform of $f(x)$ then the __power spectrum__ is $P(f(x)) = \langle \overline{F(k)}~F(k)\rangle$. 
 - The power spectrum provides an estimate of the noise matrix. 

#### (3) 
Smooth the power spectrum to manage spectral artifacts that present themselves as sharp spikes in the data. 

 - Smoothing is performed by convolving with a __smooth function__. In the most general case, a function $f:\mathbb{R}\to\mathbb{R}$ is said to be smooth at a point $x$ in its domain if it is smooth in an open neighbourhood $U$ of $x$. This in turn requires that on $U$, $\frac{d^{n}f}{dx^{n}}$ exists for all $n \in \mathbb{N}$. 
 - In this code, we smooth by using a Gaussian template function $g(x) = \exp\Big{(}-\frac{x^{2}}{2\sigma^{2}}\Big{)}$ for $x \in [-1,1]$. To actually carry out the smoothing, we effectively work with $\mathfrak{g}(n) = g(n),~n\in[0,\tfrac{N}{2}]$ and $\mathfrak{g}(n) = g(n-N),~n\in[\tfrac{N}{2}+1, N-1]$ for non-negative integers $n$. 

#### (4) 
__Whiten__ both the strain data and the template ("chirp") function that will be fitted. This is done by calculating the estimate of the inverse noise matrix, $({\langle \overline{F(k)}~F(k)\rangle}_{\text{smoothed}})^{-1}$. Then we calculate $N^{-\tfrac{1}{2}} = \sqrt{N^{-1}}$ and find $N^{-\tfrac{1}{2}}\mathbf{d}$ as well as $N^{-\tfrac{1}{2}}A$.

#### (5) 
Fit the whitened template ("chirp") function to the whitened strain data and find the linear least-squares best fit to the data. 

 - Recall that for a template $A$ with parameters $\mathbf{m}$ and a data set $\mathbf{d}$ such that $\langle\mathbf{d}\rangle = A\mathbf{m}$, the choice of $\mathbf{m}$ that minimizes $\chi^{2}$ is $\mathbf{m} = (A^{T}N^{-1}A)^{-1}A^{T}N^{-1}\mathbf{d}$.
 
### Part B: Using Python to implement the noise matrix estimate and fit the data for a gravitational wave

Below is a script that attempts to realize the blueprint prescribed in __Part A__. 

#### Reading in the LIGO data 

I make use of the script made for our class `simple_read_ligo.py` to handle the reading of the h5py files. My code reads the `BBH_events_v3.json` file to get the correct event name, file names for the Hanford and Livingston data, and the separate templates for Hanford and Livingston. 

#### Helper functions 

There are two helper functions that I based on those we saw developed in class. I call them `smoothing_interval` and `make_smooth_gauss`. 

`smoothing_interval` redistributes the range of a function to produce a smoothing function on $[0,N-1] for $N$ data points. 

`make_smooth_gauss` smooths a data set by convolving with a Gaussian that has been modified by `smoothing_interval`. 

#### Windowing 

I experiment with three different windows in my code. 

(1) $w_{1}(x) = (1-x^{2})\exp\Big{(}-\frac{x^{2}}{2\sigma^{2}}\Big{)}$

(2) Nutall window, from https://en.wikipedia.org/wiki/Window_function. $w_2(x) = a_{0}- a_{1}\cos(\frac{2\pi x}{N-1}) + a_{2}\cos(\frac{4\pi x}{N-1}) - a_{3}\cos(\frac{6\pi x}{N-1})$. 

$a_{0} = 0.355768, ~a_{1} = 0.487396, ~a_{2} = 0.144232, ~a_{3} = 0.012604$. 

(3) Blackman-Nutall window, from https://en.wikipedia.org/wiki/Window_function. $w_{3}(x) = a_{0}- a_{1}\cos(\frac{2\pi x}{N-1}) + a_{2}\cos(\frac{4\pi x}{N-1}) - a_{3}\cos(\frac{6\pi x}{N-1})$. 

$a_{0} = 0.3635819, ~a_{1} = 0.4891775, ~a_{2} = 0.1365995, ~a_{3} = 0.0106411$.

The choice of (1) is rather arbitrary - it is easy to think of a function that vanishes at $x = -1$ and $x = 1$ smoothly without the derivatives also vanishing there and I thought it would be interesting to test the utility of this function as a window. It performed poorly for these data sets. 

The choice of (2) and (3) is based on their Fourier transforms having suppressed side lobes. After looking at the data, this seemed like an attractive feature when considering that the noise is large on the boundaries of frequency space. 

#### Smoothing 

I chose to use a Gaussian template function for smoothing. Among its surprising properties, the function $g(x) = \frac{1}{\sqrt{2\pi\sigma^{2}}}\exp\Big{[}-\frac{(x-\mu)^{2}}{2\sigma^{2}}\Big{]}$ happens to belong to the class of Schwartz functions $S(\mathbb{R})$. (This class is invariant under the Fourier transform.) When convolved with other functions it yields particularly nice results and it is easy to work with analytically. 

In [221]:
import numpy as np 
import math 

def smoothing_interval(n):
    # Takes an integer n 
    # Creates a vector of numbers 1,2,...,n 
    # Forces the last n/2 entries to be equal to the first n/2 subtract n 
    
    x     = np.arange(n)
    i     = int(n/2) 
    x[i:] = x[i:] - n 
    
    assert(x[-1] == -1) # force the final entry in the vector to be -1, just in case it isn't 
    
    return 1.0*x # need it to be a float     
        
def make_smooth_gauss(data, fwhm):
    
    # This function will convolve `data` with a Gaussian by first computing 
    # the FT of each and then taking their product; finally, taking the inverse FT 
    # will map back to the initial domain and preserve normalizations (by Parseval's theorem)
    
    n     = len(data)
    x     = smoothing_interval(n)
    sigma = fwhm/(2*np.sqrt(2*np.log(2))) # formula for finding sigma from fwhm  
    
    smoothie = np.exp(-(x**2)/(2*(sigma**2))) 
    smoothie = smoothie/(smoothie.sum())
    
    ft_data     = np.fft.rfft(data)
    ft_smoothie = np.fft.rfft(smoothie)
    
    ft_product  = ft_data*ft_smoothie 
    
    return np.fft.irfft(ft_product, n=n)
    

In [331]:
import simple_read_ligo as rl

import numpy as np
import math
import h5py
import pylab
import glob
import json

from pprint import pprint

from pylab import *

from matplotlib import pyplot as plt

plt.ion()
plt.clf()
plt.figure(figsize=(15,10))

#------------------------------------------------------# 
# Load the data from the .json file 
#------------------------------------------------------#

nevents = 4

with open("BBH_events_v3.json") as f:
    events = json.load(f)
    
event_names = ["GW150914", "LVT151012", "GW151226", "GW170104"]

#------------------------------------------------------# 
# Loop over the events  
#------------------------------------------------------#

for i in range(nevents):
    
    print("Processing event #" + str(i+1) + ":")
    print("------------------")
    
    # Get the event dict from the .json file 
    event_dict = events[event_names[i]]
    
    fn_H1 = event_dict["fn_H1"]    # Hanford file name
    fn_L1 = event_dict["fn_L1"]    # Livingston file name
    
    # Read the files 
    print("Reading file " + fn_H1 + " ...")
    strain_H, dt_H, utc_H = rl.read_file(fn_H1)
    print("Reading file " + fn_L1 + " ...")
    strain_L, dt_L, utc_L = rl.read_file(fn_L1)
    
    s_H = len(strain_H)
    s_L = len(strain_L)
    #print("s_H == s_L: " + str(np.allclose(s_H, s_L))) # -> TRUE
    #print("t_H == t_L: " + str(np.allclose(t_H, t_L))) # -> TRUE 
    
    n  = s_H
    dt = dt_H
    t  = np.arange(n)*dt 
    
    label_H = "H" + str(i+1) + "_" + event_names[i]
    label_L = "L" + str(i+1) + "_" + event_names[i]
    
    """ Plotting
        --------   
    
    plt.loglog(t, strain_H, linewidth=2, label=label_H)
    plt.loglog(t, strain_L, linewidth=2, label=label_L)
    if i==3: 
        plt.title("Advanced LIGO: measured strain for 4 events from Hanford (H) and Livingston (L)")
        plt.xlabel("Log$_{10}$(Time since event [s])")
        plt.ylabel("Log$_{10}$(Strain)")
        legend(loc="lower left")
        name = "plot_all_strains.png"
        pylab.savefig(name)
        print("Saved figure " + name)
    """
    
    
    # Get the template functions 
    fn_template = event_dict["fn_template"]
    print("Template: " + fn_template)
    temp_H, temp_L = rl.read_template(fn_template)     
    
    # Try some window functions 
    
    #x = np.linspace(-1, 1, n)
    x = np.arange(n)

    #---------------#
    # window option 1
    #---------------#
    
    # Reference: a function of my own creation; it goes to a very small value at -1, 1; the 
    # 1st derivative goes to a larger nonzero value but both can be adjusted by sigma. 
    
    sigma = 0.3 # window parameter 
    w1 = (-x**2+1)*np.exp(-x**2/(2*sigma**2)) 
    
    # N.B. Post-processing note: too much unwhitened noise is leftover on the ends of the time 
    # interval, the signal to noise ratio would be low! 
    
    # --------------# 
    # window option 2 
    # --------------#
    
    # Reference: https://en.wikipedia.org/wiki/Window_function "Nutall window"
    
    a0 = 0.355768 
    a1 = 0.487396
    a2 = 0.144232
    a3 = 0.012604
    pi = math.pi 
    
    w2 = a0 - a1*np.cos((2*pi*x)/(n-1)) + a2*np.cos((4*pi*x)/(n-1)) - a3*np.cos((6*pi*x)/(n-1))
    
    # N.B. Post-processing note: large artifacts leftover at the lower bound of the time interval 
    
    #---------------#
    # window option 3
    #---------------#
    
    # Reference: https://en.wikipedia.org/wiki/Window_function "Blackman-Nutall window"
    
    a0 = 0.3635819 
    a1 = 0.4891775
    a2 = 0.1365995
    a3 = 0.0106411
    
    w3 = a0 - a1*np.cos((2*pi*x)/(n-1)) + a2*np.cos((4*pi*x)/(n-1)) - a3*np.cos((6*pi*x)/(n-1))
    
    # N.B. Post-processing note: this window seems to work the best; there are some strange 
    # artifacts at the ends of the intervals similar to what w2 did, but they are much smaller 
    
    #---------------#
    
    # Choose a window now 
    #w = w1 
    #w = w2
    w = w3
    
    #plt.plot(w)
    #plt.show()
    
    ft_strain_H = np.fft.rfft(strain_H*w)
    ft_strain_L = np.fft.rfft(strain_L*w)
    ps_H        = (np.abs(ft_strain_H))**2  # absolute value, just as an artifact from complex case 
    ps_L        = (np.abs(ft_strain_L))**2 
    
    dnu = 1/(n*dt)
    nu  = np.arange(len(ps_H))*dnu
    
    """ Plotting
        --------
    
    plt.loglog(nu, ps_H, linewidth=2, label=label_H)
    plt.loglog(nu, ps_L, linewidth=2, label=label_L)
    if i==3: 
        plt.title("Advanced LIGO: power spectrum for 4 events from Hanford (H) and Livingston (L)")
        plt.xlabel("Log$_{10}$(Frequency [Hz])")
        plt.ylabel("Log$_{10}$(Power)")
        legend(loc="lower left")
        name = "plot_all_ps.png"
        pylab.savefig(name)
        print("<-! Saved figure " + name)
    
    """
    
    # Now smooth the power spectra 

    smooth_ps_H = make_smooth_gauss(ps_H, 30)
    smooth_ps_L = make_smooth_gauss(ps_L, 30)
    
    #plt.loglog(nu, smooth_ps_H)
    #plt.loglog(nu, smooth_ps_L)
    
    """ Plotting
        --------
        
    plt.loglog(nu, smooth_ps_H, linewidth=2, label=label_H)
    plt.loglog(nu, smooth_ps_L, linewidth=2, label=label_L)
    if i==3: 
        plt.title("Advanced LIGO: smoothed power spectrum for 4 events from Hanford (H) and Livingston (L)")
        plt.xlabel("Log$_{10}$(Frequency [Hz])")
        plt.ylabel("Log$_{10}$(Power)")
        legend(loc="lower left")
        name = "plot_all_smoothed_ps.png"
        pylab.savefig(name)
        print("<-! Saved figure " + name)
    """
    
    # Now try whitening the smoothed PS by calculating N^{-0.5}*data 
    # With stationary noise, this is easily done: 
    
    Ni_H = 1.0/smooth_ps_H   # inverse of (estimated!) noise matrix 
    Ni_L = 1.0/smooth_ps_L 
    
    # Before we apply the inverse square root to the data, we need to be careful. 
    # Some parts of the smoothed power spectrum have very sharp spectral lines. 
    # Let's remove a chunk at the lower frequency end and the higher frequency end. 
    
    # Theses dissections were decided after looking at the data to see what spectral 
    # features there are at low and high frequencies. It appears that ~< 10 Hz is 
    # very noisy and that >~ 1400 Hz has a bizarre structure that is assumed to be 
    # artificial. (The explanation of the structure is beyond the scope of this work.)
    
    Ni_H[nu <= 10]   = 0
    Ni_H[nu >= 1300] = 0
    
    Ni_L[nu <= 10]   = 0
    Ni_L[nu >= 1300] = 0
    
    whitened_ft_strain_H = np.sqrt(Ni_H)*ft_strain_H
    whitened_ft_strain_L = np.sqrt(Ni_L)*ft_strain_L
    
    whitened_strain_H = np.fft.irfft(whitened_ft_strain_H)
    whitened_strain_L = np.fft.irfft(whitened_ft_strain_L)
    
    #plt.plot(t, whitened_strain_H)
    #plt.plot(t, whitened_strain_L)
    
    """ Plotting
        --------
    
    plt.plot(t, whitened_strain_H, linewidth=2, label=label_H)
    plt.plot(t, whitened_strain_L, linewidth=2, label=label_L)
    if i==3: 
        plt.title("Advanced LIGO: whitened strain for 4 events from Hanford (H) and Livingston (L)")
        plt.xlabel("Log$_{10}$(Time since event [s])")
        plt.ylabel("Log$_{10}$(Strain)")
        legend(loc="lower left")
        name = "plot_all_whitened_strains.png"
        pylab.savefig(name)
        print("<-! Saved figure " + name)
    """
    
    # Now to fit the corresponding template for each event 
    # First get the FT of the windowed template, then whiten as above
    
    ft_temp_H          = np.fft.rfft(temp_H*w)
    whitened_ft_temp_H = ft_temp_H*np.sqrt(Ni_H)
    whitened_temp_H    = np.fft.irfft(whitened_ft_temp_H)
    
    ft_temp_L          = np.fft.rfft(temp_L*w)
    whitened_ft_temp_L = ft_temp_L*np.sqrt(Ni_L)
    whitened_temp_L    = np.fft.irfft(whitened_ft_temp_L)
    
    # Now we shift the whitened FT template along every point of the whitened FT strain 
    # and multiply them pointwise; since the Fourier transform of a convolution is the 
    # pointwise multiplication of the Fourier transforms then taking inverse Fourier 
    # transform of a pointwise multiplication yields a convolution 
    
    m_H = np.fft.irfft(whitened_ft_strain_H*np.conj(whitened_ft_temp_H))
    m_L = np.fft.irfft(whitened_ft_strain_L*np.conj(whitened_ft_temp_L))
    
    ft_m_H = np.real(np.fft.rfft(m_H))
    ft_m_L = np.real(np.fft.rfft(m_L))
    
    plt.plot(t, m_H)
    print("Saving Hanford plot...")
    pylab.savefig("plot_H" + str(i+1) + "_" + event_names[i] + ".png")
    plt.clf()
    
    plt.plot(t, m_L)
    print("Saving Livingston plot...\n")
    pylab.savefig("plot_L" + str(i+1) + "_" + event_names[i] + ".png")
    plt.clf()
    
    plt.plot(nu, ft_m_H)
    print("Saving Hanford plot...")
    pylab.savefig("plot_ft_H" + str(i+1) + "_" + event_names[i] + ".png")
    plt.clf()
    
    plt.plot(nu, ft_m_L)
    print("Saving Livingston plot...\n")
    pylab.savefig("plot_ft_L" + str(i+1) + "_" + event_names[i] + ".png")
    plt.clf()
    
    
    print("************")
    print("Calculations")
    print("************\n")
    
    amp_H = np.max(np.abs(m_H))
    amp_L = np.max(np.abs(m_L))
    
    # We estimate the noise in the signal by taking the 
    # standard deviation of points in the neighbourhood [0:3000] of the signal spike
    
    err_amp_H = np.std(m_H[0:3000]) 
    err_amp_L = np.std(m_L[0:3000])
    
    SNR_H = amp_H/err_amp_H
    SNR_L = amp_L/err_amp_L 
    
    combo = np.abs(m_H) + np.abs(m_L)
    
    amp_combo     = np.max(np.abs(combo))
    err_amp_combo = np.std(combo[0:3000])
    SNR_combo     = amp_combo/err_amp_combo 
    
        
    print("SNR for H:            " + str(SNR_H))
    print("SNR for L:            " + str(SNR_L))
    print("SNR for combined H+L: " + str(SNR_combo))
    
    # Now for the last part, we want to find the frequency such that half the weight 
    # comes from higher frequencies and the other half of the weight comes from lower 
    # frequencies 
    
    mmT_H = (np.dot(ft_m_H, ft_m_H.transpose()))
    mmT_L = (np.dot(ft_m_L, ft_m_L.transpose()))
    
    avg_mmT_H = np.average(mmT_H)
    avg_mmT_L = np.average(mmT_L)
    
    #print("<mmT> for H: " + str(avg_mmT_H))
    #print("<mmT> for L: " + str(avg_mmT_L))
    
    summand = 0 
    index   = -1
    j = 0
    
    #print("t.size  = " + str(t.size))
    #print("nu.size = " + str(nu.size))
    
    while summand < 0.5*avg_mmT_H: 
        summand += ft_m_H[j]**2
        index = j
        j += 1
    
    print("Half-sum index for H is     " + str(index))
    print("Half-sum frequency for H is " + str(nu[index]) + " Hz")
    
    summand = 0 
    index   = -1
    j = 0
    
    while summand < 0.5*avg_mmT_L: 
        summand += ft_m_L[j]**2
        index = j
        j += 1
    
    print("Half-sum index for L is     " + str(index))
    print("Half-sum frequency for L is " + str(nu[index]) + " Hz")
    
    """
    t_test = np.abs(amp_H - amp_L)/np.sqrt(err_amp_H**2 + err_amp_L**2)
    
    amp_avg = -1
    err_avg = -1
    
    if t_test <= 2:
        print("Consistent results.")
        amp_avg = (err_amp_L**2*amp_H + err_amp_H**2*amp_L)/(err_amp_H**2 + err_amp_L**2)
        err_avg = np.sqrt(err_amp_H**2 + err_amp_L**2)
    else:
        print("Inconsistent results.")
        amp_avg = (amp_H+amp_L)/2
        err_avg = (np.abs(amp_H - amp_L) + err_amp_H + err_amp_L)/2
        
    print("Averaged amplitude: " + str(amp_avg))
    print("Averaged error:     " + str(err_avg))
    
    SNR_combined = amp_avg/err_avg
    
    print("Combined SNR:       " + str(SNR_combined))
    """
    print ("--------//--------\n\n\n")
    

Processing event #1:
------------------
Reading file H-H1_LOSC_4_V2-1126259446-32.hdf5 ...
Reading file L-L1_LOSC_4_V2-1126259446-32.hdf5 ...
Template: GW150914_4_template.hdf5
Saving Hanford plot...
Saving Livingston plot...

Saving Hanford plot...
Saving Livingston plot...

************
Calculations
************

SNR for H:            12.054440674239627
SNR for L:            9.92573069171091
SNR for combined H+L: 12.514352426773288
Half-sum index for H is     3415
Half-sum frequency for H is 106.71875 Hz
Half-sum index for L is     3663
Half-sum frequency for L is 114.46875 Hz
--------//--------



Processing event #2:
------------------
Reading file H-H1_LOSC_4_V2-1128678884-32.hdf5 ...
Reading file L-L1_LOSC_4_V2-1128678884-32.hdf5 ...
Template: LVT151012_4_template.hdf5
Saving Hanford plot...
Saving Livingston plot...

Saving Hanford plot...
Saving Livingston plot...

************
Calculations
************

SNR for H:            5.918617397297965
SNR for L:            5.2062965269

<Figure size 432x288 with 0 Axes>

<Figure size 1080x720 with 0 Axes>